In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
# Load the dataset
file_path = "Data\winequality-red.csv"  # Update if needed
df = pd.read_csv(file_path)

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Saian\AppData\Local\Temp\ipykernel_6308\1186338799.py:2: SyntaxWarning: invalid escape sequence '\w'
  file_path = "Data\winequality-red.csv"  # Update if needed


In [7]:
# Get pH column
pH = df['pH'].dropna()

# Print mean pH for sanity
mean_pH = np.mean(pH)
print(f"Mean pH: {mean_pH}")

Mean pH: 3.3111131957473416


In [4]:

# Count how many entries
nEntries = pH.count()
print(f"Entries: {nEntries}")


Entries: 1599


In [11]:
# Perform a t-test against the hypothesised mean
mu = 3.4
t_stat, p_value = stats.ttest_1samp(pH, popmean=mu)

# Display the results
print("One-sample t-test against a hpothesised mean")
print("t_stat = ", t_stat)
print("p_value = ", p_value)

# Check against the significance level
alpha = 0.05
if p_value < alpha:
    print("\nConclusion: Reject the null hypothesis (H₀).")
else:
    print("\nConclusion: Fail to reject the null hypothesis (H₀).")

One-sample t-test against a hpothesised mean
t_stat =  -23.022490434071052
p_value =  1.5992055282971605e-101

Conclusion: Reject the null hypothesis (H₀).


In [17]:
# Manual Approach
std_high_quality = np.std(high_quality)
mean_high_quality = np.mean(high_quality)

std_low_quality = np.std(low_quality)
mean_low_quality = np.mean(low_quality)

# Calculate SE of difference - population based
SE_diff = np.sqrt(np.square(std_high_quality) / nHighQuality + np.square(std_low_quality) / nLowQuality)
print(f"Population-based Standard Error of the Difference = {SE_diff:3f}")

# Calculate Z
Z = ((mean_high_quality - mean_low_quality) - 0) / SE_diff
print(f"Z-score = {Z:3f}")

#  Calculate p-value - p >= |Z|
p_value_manual = stats.norm.sf(abs(Z)) * 2 # Two-sided
print(f"p-value = {p_value_manual}")


Population-based Standard Error of the Difference = 0.526229
Z-score = -2.460317
p-value = 0.013881421244196395


In [18]:
# Power Analysis - Analytical Approach

from statsmodels.stats.power import TTestIndPower

# Instantiate TTestIndPower class
analysis = TTestIndPower()

# Anticipated difference
difference = 1.0

# Assumed standard deviation
std = 1.0

# Cohen's d: Effect Size = (difference / std)
effect_size = difference / std

# Set significance level and power
alpha = 0.05
power = 0.90

# Solve for sample size per group
sample_size = analysis.solve_power(effect_size=effect_size,
                                   alpha=alpha,
                                   power=power,
                                   alternative='two-sided')

# Display the result
print("Required sample size per group = ", sample_size)

#  Suppose we fix n per group, find the power
nFix = 20
my_power = analysis.solve_power(effect_size=effect_size,
                          alpha=alpha,
                          nobs1=nFix,
                          alternative='two-sided')

# Display the result
print(f"Power with n = {nFix} per group = {my_power:3f}")

Required sample size per group =  22.021089673603985
Power with n = 20 per group = 0.868953


In [19]:
# Power Analysis - Simulated Approach

# Build a function for simulating power
def simulate_power(n=20, true_diff=1.0, sigma=1.0, alpha=0.05, reps=5000):

    # Estimate power for a two-sample t-test (equal variances) via simulation

    np.random.seed(42)
    rejects = 0

    for _ in range(reps):

        group1 = np.random.normal(loc=0, scale=sigma, size=n)
        group2 = np.random.normal(loc=true_diff, scale=sigma, size=n)

        # Apply a two-sample t-test
        t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=True)

        # Check if p_val < alpha, reject H0 if so
        if p_val < alpha:

            # Increment rejects
            rejects += 1

    # Fraction of rejections approximates power
    power_est = rejects / reps
    return power_est


#  Call the function
power_sim = simulate_power(n=20, true_diff=1.0, sigma=1.0, alpha=0.05, reps=5000)

#  Display the result
print(f"Simulated power estimate gives power = {power_sim:3f}")

Simulated power estimate gives power = 0.873200
